<a href="https://colab.research.google.com/github/Ele975/LLM_from_scratch/blob/development/LLM_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install tiktoken

In [37]:
import re
import urllib.request
import tiktoken
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

Download small dataset for training

In [38]:
url = ("https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")

file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7dea7e65b610>)

In [39]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
print('Total number of chars:', len(raw_text))
print(raw_text[:99])

Total number of chars: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# Preprocessing

## Text tokenization (not necessary, demonstrative purpose only)

In [40]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# remove white spaces
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])


['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Vocabulary creation (not necessary, demonstrative purpose only)

In [41]:
# generate ordered set of unique tokens
all_words = sorted(set(preprocessed))
# add token for unkown words (not in the vocab) and for termination of documents when concatenation (inputs are concatenated and model should distinguish them)
all_words.extend(["<|endoftext|>", "<|unk|>"])
print(len(all_words))
print(all_words[:30])
print(all_words[-5:])


1132
['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft', 'Destroyed']
['younger', 'your', 'yourself', '<|endoftext|>', '<|unk|>']


In [42]:
# generate vocabulary
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
  if i < 50:
    print(item)
  else:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)


Class to convert from token to ID and vice-versa

In [43]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    # inverse vocabulary permitting to map from int to token
    self.int_to_str = {integer:token for token, integer in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    # unkown tag if token not in vocab
    preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    # remove unnecessary spaces
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text

Usage example

In [44]:
# tokenizer = SimpleTokenizerV1(vocab)
# text = """"It's the last he painted, you know,"
#        Mrs. Gisburn said with pardonable pride."""

# ids = tokenizer.encode(text)
# print(ids)

# print(tokenizer.decode(ids))
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


BPE tokenizer

In [45]:
tokenizer = tiktoken.get_encoding("gpt2")

## Dataset class and dataloader
Through sliding window with parameters as context size (length) and stride

In [46]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [47]:
enc_sample = enc_text[:50]

In [48]:
# elements in the input (window' size)
context_size = 4

dataset class

In [49]:
class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+1+max_length]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]


Dataloader

In [50]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(
      dataset,
      batch_size = batch_size,
      shuffle = shuffle,
      drop_last = drop_last,
      num_workers = num_workers
  )

  return dataloader

Print first batch example

In [51]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


## Token embedding
Transform tokens IDs into embedding vectors (embedding can be optimized). Necessary since vectors are a continuous representation and neural networks use backward propagation for training. The embedding layer has dimension (vocab_size x embedding_dim), since for each word in the covabulary we'll have an embedding vector of size embedding_dim. This embedding matrix is optimized during the training of the model as part of the training itself. Given then the token ID, it is possible to retrieve from the embedding matrix the embedding vector for that specific token.

In [52]:
# size vocabulary BPE tokenizer
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

Instantiate data loader

In [53]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

# use embedding layer to embed the tokens inside the first batch
# retrieve embedding vectors given IDs
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
torch.Size([8, 4, 256])


Add absolute embedding approach

In [54]:
context_length = max_length

pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
# indices for positions 0,1,2 .. context_length - 1, i.e. 0, 1, 2 .. context_length - 1
# retrieve embedding vectors given IDs
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


Add positional embeddings to basic embeddings = input embedding

In [55]:
input_embeddings = token_embeddings + pos_embeddings

# Attention mechanism

## 1. Simplified version

Embedding vectors of 'Your journey starts with one step'

In [56]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
[0.57, 0.85, 0.64], # starts
[0.22, 0.58, 0.33], # with
[0.77, 0.25, 0.10], # one
[0.05, 0.80, 0.55]] # step
)


Computation attention score for a specific embedding vector (token) with respect to all the others -> dot product between selected token (embedded query token) and all the other embedding vectors

In [57]:
# computing all attention score for all inputs, i.e. 6 values for each input
attn_scores = inputs @ inputs.T
print(attn_scores)


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


normalized attention scores, i.e. AS[2]/tot(AS) or with Softmax (more used since better managing of extreme values)

In [58]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print("Attention weights:", attn_weights)
print('All row sums:', attn_weights.sum(dim=-1))

Attention weights: tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


Attention weights: sum of all input embedding multiplied with their attention score

In [59]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self-attention (with trainable parameters)

In [60]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

Matrices (query, key, value)

In [61]:
torch.manual_seed(123)
# requires_grad should be true if we really use them since they must be updated during training
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)


In [62]:
# pytorch translate automatically x_2 to adjust the dimensions
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


Get all keys and values for all input

In [63]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


Attention score: dot product between query vector and key vector, i.e. attention score 2_2: query2.key2

In [64]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


Attention score of all inputs with respect to input 2 (i.e. using query 2)

In [65]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


Attention weights -> normalization and softmax -> divide attention scores by dividing them by the square root of the embedding dimension of the keys

In [66]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2/d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


Context vector for input 2 -> weighted sum over the value vectors

In [67]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


Compact implementation using Python class for each input

In [78]:
class SelfAttention_V1(nn.Module):
  def __init__(self, d_in, d_out, qkv_bias=False):
    # call superclass init since internal initialization logic should be executed for nn.Module
    super().__init__()
    # nn.Linear has optimized weight initialization scheme
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

  # x = all inputs
  def forward(self, x):
    keys = self.W_key(x)
    values = self.W_value(x)
    queries = self.W_query(x)

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
    context_vec = attn_weights @ values
    return context_vec

How to use it (example)

In [83]:
torch.manual_seed(789)
sa_v1 = SelfAttention_V1(d_in, d_out)
# returns the context vectors
print(sa_v1(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## Causal attention